## Install

In [ ]:
!pip install clip-retrieval img2dataset

## Get some image urls

In [ ]:
#!echo 'https://placekitten.com/200/305' >> myimglist.txt
#!echo 'https://placekitten.com/200/304' >> myimglist.txt
#!echo 'https://placekitten.com/200/303' >> myimglist.txt

## Download the image urls

In [ ]:
#!img2dataset --url_list=myimglist.txt --output_folder=image_folder --thread_count=64 --image_size=256

## Produce embeddings

In [ ]:
#!clip-retrieval inference  --input_dataset image_folder --output_folder embedding_folder

In [ ]:
#!clip-retrieval inference  --input_dataset /ssd/mlrom/Data/coco/cocodataset/images/train2017 --output_folder /home/junjie/clip_test/embedding_coco_train2017

In [ ]:
#!ls -R embedding_folder

In [ ]:
# python version of use clip_retrieval inference
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3,4,5,6"

from clip_retrieval.clip_inference.main import main

main(
    input_dataset=["/ssd/mlrom/Data/coco2017/train2017"], 
    output_folder="/home/junjie/test/multitaskparalleltest/emb_coco_1024b_b32_10000writeb_4gpu", 
    batch_size=1024, 
    clip_model="ViT-B/32",#"ViT-B/32", "ViT-L/14"
    use_jit=True,
    distribution_strategy="sequential",
    enable_text=False,
    enable_image=True,
    enable_metadata=False
)

In [ ]:
# read stats from inference
import os, glob, json
#output_folder= "/home/junjie/clip_test/embedding_coco_train2017"
#output_folder= "/home/junjie/clip_test/embedding_coco_train2017_pyapi"
output_folder= "/home/junjie/clip_test/embedding_coco_train2017_pyapi_Vitl14_nojit"
#output_folder= "/home/junjie/clip_test/embedding_coco_train2017_pyapi_Vitl14_pyspark"
jfile_path = os.path.join(output_folder, "stats")
jfile = glob.glob(jfile_path+"/*.json")
#print(len(jfile))

with open(jfile[0]) as stats_file:
    stats = json.load(stats_file)
    print(stats)

## Produce knn indices

In [ ]:
# index Vit-L/14 Sequential
#!clip-retrieval index --embeddings_folder=/home/junjie/clip_test/embedding_coco_train2017_pyapi_Vitl14_pyspark --index_folder=/home/junjie/clip_test/index_coco_train2017_pyapi_Vitl14_pyspark

In [ ]:
# index no jit
#!clip-retrieval index --embeddings_folder=/home/junjie/clip_test/embedding_coco_train2017_pyapi_pyspark --index_folder=/home/junjie/clip_test/index_coco_train2017_pyapi_pyspark

In [ ]:
# python api build index
from clip_retrieval.clip_index import clip_index

clip_index(
    embeddings_folder="/home/junjie/test/embedding_script_multi",
    index_folder="/home/junjie/test/index_script_multi",
    max_index_memory_usage="4G",
    current_memory_available="16G",
    copy_metadata=True,
    nb_cores=None
)

|model|JIT|distribution|read Duration|inference_duration|write_duration|total_duration|sample_count|size|
|---|---|---|---|---|---|---|---|---|
|ViT-B/32|yes|sequential|37.42|164.62|0.02|202.06(3.3 min)|118287|121126016 (116 Mb)|
|ViT-B/32|no|sequential|15.87|186.23|0.02|202.13|118287|121126016|
|ViT-B/32|yes|pyspark|32.67|166.09|0.03|198.80|118287|121126016|
|ViT-L/64|yes|sequential|6.10|3220.61|0.05|3226.76(53.8 min)|118287|181688960 (174 Mb)|
|ViT-L/64|no|sequential|5.85|3392.02|0.05|3397.92|118287|181688960|
|ViT-L/64|yes|pyspark|6.07|3220.89|0.05|3227.01|118287|181688960|


## Use the index to get a subset of files

In [ ]:
#!clip-retrieval filter --query "cat" --output_folder "cat/" --indice_folder "index_folder"

In [ ]:
#!ls

In [ ]:
#ls -R cat

In [ ]:
#from IPython.display import Image
#Image(filename='cat/000000000.jpg') 

## Run the knn service backend

In [ ]:
%%bash
echo '{"coco_train2017": "/home/junjie/clip_test/index_coco_train2017_pyapi", "coco_train2017_nojit": "/home/junjie/clip_test/index_coco_train2017_pyapi_nojit", "coco_train2017_pyspark": "/home/junjie/clip_test/index_coco_train2017_pyapi_pyspark"}' > /home/junjie/clip_test/indices_paths_B_32.json
echo '{"coco_train2017_vitl14": "/home/junjie/clip_test/index_coco_train2017_pyapi_Vitl14", "coco_train2017_vitl14_nojit": "/home/junjie/clip_test/index_coco_train2017_pyapi_Vitl14_nojit", "coco_train2017_vitl14_pyspark": "/home/junjie/clip_test/index_coco_train2017_pyapi_Vitl14_pyspark"}' > /home/junjie/clip_test/indices_paths_L_14.json
#npm install -g localtunnel

In [ ]:
# after running the next cell, visit the localtunnel url once then go to
# https://rom1504.github.io/clip-retrieval/?back=<your local tunnel url here>

In [ ]:
#from threading import Thread
#
#def app():
#  !clip-retrieval back --port 1234 --indices-paths indices_paths.json
#
#if __name__ == '__main__':
#    t1 = Thread(target = app)
#    a = t1.start()
#    !lt --port 1234